<a href="https://colab.research.google.com/github/steveSchwering/lichtheim_memory/blob/main/lichtheim-memory/lichtheim_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## General compute

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/dissertation/lichtheim-memory'
%ls

/content/drive/My Drive/Colab Notebooks/dissertation/lichtheim-memory
_deprecated/            LichtheimMemory.py  lm_logging.py
helperfunctions.py      lm_behavior.py      model_info/
language/               lm_dataloading.py   __pycache__/
lichtheim-memory.ipynb  LMInterface.py      random_list_generator.py


In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'Current device: {torch.device(device)}')
print(f'Do we have a GPU available?: {torch.cuda.is_available()}')
if torch.cuda.is_available():
  print(f'What is the current device on which we are loading the model?: {torch.cuda.current_device()}')
  print(f'What is name of the current device?: {torch.cuda.get_device_name(0)}')

Current device: cpu
Do we have a GPU available?: False


In [ ]:
torch.set_default_dtype(torch.float64)

## Reading in the artificial language and representations

The LMLInterface object is responsible for translating between sentences and the model.

In [ ]:
import LMInterface

from pathlib import Path

# Get language information
language_name = 'datives'
language_dir = Path.cwd().joinpath(f'language/{language_name}')

interface = LMInterface.LMInterface(language_dir = language_dir,
                                    phonology_filestem = 'phonology_artificial')

# Generate sentences
sentences_dir = Path.cwd().joinpath(f'language/{language_name}/all_sentences.tsv')
sentences = LMInterface.utterances_from_file(sentences_dir, sep = "\t")

# Extract words
words = list(interface.word_to_semantics.keys())

In [ ]:
print(type(sentences[0]))

<class 'pandas.core.series.Series'>


## The Lichtheim-Memory model

In [ ]:
import LichtheimMemory    # Lichtheim-memory model

import lm_dataloading     # Defines splitting train from test
import lm_behavior        # Training functions and wrappers
import lm_logging         # Functions to work with model output

In [ ]:
from importlib import reload

reload(LMInterface)
reload(LichtheimMemory)
reload(lm_dataloading)
reload(lm_behavior)
reload(lm_logging)

<module 'lm_logging' from '/content/drive/MyDrive/Colab Notebooks/dissertation/lichtheim-memory/lm_logging.py'>

## Instantiation and training functions

In [ ]:
def instantiate_model_and_environment(words, utterances, tt_split, seed, interface,
                                      phh1_size, wsh1_size, wsh2_size, ssh1_size, ssh2_size, ssh3_size,
                                      lr, weight_decay, lr_steprate, lr_gamma,
                                      sample_tt_probability = True, bin_tt_label = 'probability',
                                      save_init_model = True):
  """
  """
  # Generate training and testing sets for the model. Save them to disk.
  dl_words, dl_sent_train, dl_sent_test, sent_train, sent_test = lm_dataloading.inst_training_env(words = words,
                                                                                                  utterances = sentences,
                                                                                                  tt_split = tt_split,
                                                                                                  interface = interface,
                                                                                                  sample_tt_probability = sample_tt_probability,
                                                                                                  bin_tt_label = bin_tt_label,
                                                                                                  seed = seed)

  # Generate model, along with optimizer and scheduler
  LM, optimizer, scheduler = LichtheimMemory.inst_LM(interface = interface,
                                                      phh1_size = phh1_size, wsh1_size = wsh1_size, wsh2_size = wsh2_size,
                                                      ssh1_size = ssh1_size, ssh2_size = ssh2_size, ssh3_size = ssh3_size,
                                                      lr = lr, weight_decay = weight_decay, lr_steprate = lr_steprate, lr_gamma = lr_gamma,
                                                      seed = seed)

  # Log the initialization of the model
  if save_init_model:
    lm_logging.log_model_init(model = LM,
                              language_dir = interface.language_dir,
                              seed = seed,
                              sent_train = sent_train,
                              sent_test = sent_test)

  # Save the initialized model
  training_meta_info = {'language' : interface.language_dir, 'seed' : seed, 'sample_tt_probability' : sample_tt_probability, 'bin_label' : bin_tt_label,
                        'lr' : lr, 'weight_decay' : weight_decay, 'lr_steprate' : lr_steprate, 'lr_gamma' : lr_gamma,
                        'word_epochs_per_sentence_epoch' : word_epochs_per_sentence_epoch, 'word_epochs_decay_stepsize' : word_epochs_decay_stepsize, 'word_epochs_decay_steprate' : word_epochs_decay_steprate}
  if save_init_model:
    lm_logging.log_model_state(model = LM,
                               trained_epochs = "init",
                               training_meta_info = training_meta_info)

  return dl_words, dl_sent_train, dl_sent_test, sent_train, sent_test, LM, optimizer, scheduler, training_meta_info

In [ ]:
def inst_and_train_model(words, utterances, tt_split, interface, seed,
                         phh1_size, wsh1_size, wsh2_size, ssh1_size, ssh2_size, ssh3_size,
                         lr, weight_decay, lr_steprate, lr_gamma,
                         num_train_epochs,
                         word_epochs_per_sentence_epoch, word_epochs_decay_stepsize, word_epochs_decay_steprate,
                         report_every, summarize_every, log_every,
                         sample_tt_probability = True, bin_tt_label = 'structure',
                         save_init_model = True):
  """
  """
  dl_words, dl_sent_train, dl_sent_test, sent_train, sent_test, LM, optimizer, scheduler, training_meta_info = instantiate_model_and_environment(words = words, utterances = utterances, tt_split = tt_split, interface = interface, seed = seed,
                                                                                                                                                 phh1_size = phh1_size, wsh1_size = wsh1_size, wsh2_size = wsh2_size, ssh1_size = ssh1_size, ssh2_size = ssh2_size, ssh3_size = ssh3_size,
                                                                                                                                                 lr = lr, weight_decay = weight_decay, lr_steprate = lr_steprate, lr_gamma = lr_gamma,
                                                                                                                                                 sample_tt_probability = sample_tt_probability, bin_tt_label = bin_tt_label,
                                                                                                                                                 save_init_model = save_init_model)

  # Train the model
  all_loss_word, all_loss_sentence, training_meta_info = lm_behavior.interleave_training(model = LM,
                                                                                          dataloader_sentences = dl_sent_train,
                                                                                          dataloader_words = dl_words,
                                                                                          optimizer = optimizer,
                                                                                          scheduler = scheduler,
                                                                                          num_epochs = num_train_epochs,
                                                                                          report_every = report_every,
                                                                                          summarize_every = summarize_every,
                                                                                          log_every = log_every,
                                                                                          training_meta_info = training_meta_info,
                                                                                          word_epochs_per_sentence_epoch = word_epochs_per_sentence_epoch,
                                                                                          word_epochs_decay_stepsize = word_epochs_decay_stepsize,
                                                                                          word_epochs_decay_steprate = word_epochs_decay_steprate)

  # Final log of model to ensure it is saved
  # NOTE USE OF num_train_epochs - 1 TO CALCULATE TRAINED_EPOCHS NUM
  lm_logging.log_model_state(model = LM,
                             trained_epochs = num_train_epochs - 1,
                             training_meta_info = training_meta_info)

  return all_loss_word, all_loss_sentence, training_meta_info

## Train

In [ ]:
tt_split = 0.75
seed = 128
sample_tt_probability = True
bin_tt_label = 'structure'

phh1_size = 200
wsh1_size = 100
wsh2_size = 100
ssh1_size = 100
ssh2_size = 200
ssh3_size = 100

lr = 0.50
weight_decay = 10e-6
lr_steprate = 50
lr_gamma = 0.1

word_epochs_per_sentence_epoch = 3
word_epochs_decay_stepsize = 1
word_epochs_decay_steprate = 10

num_train_epochs = 200

log_every = 1
report_every = num_train_epochs + 1
summarize_every = 5

o = inst_and_train_model(words = words, utterances = sentences, tt_split = tt_split, interface = interface, seed = seed, sample_tt_probability = sample_tt_probability, bin_tt_label = bin_tt_label,
                          phh1_size = phh1_size, wsh1_size = wsh1_size, wsh2_size = wsh2_size, ssh1_size = ssh1_size, ssh2_size = ssh2_size, ssh3_size = ssh3_size,
                          lr = lr, weight_decay = weight_decay, lr_steprate = lr_steprate, lr_gamma = lr_gamma,
                          word_epochs_per_sentence_epoch = word_epochs_per_sentence_epoch, word_epochs_decay_stepsize = word_epochs_decay_stepsize, word_epochs_decay_steprate = word_epochs_decay_steprate,
                          num_train_epochs = num_train_epochs,
                          log_every = log_every, report_every = report_every, summarize_every = summarize_every)

Epoch 5
	Repetition -- Loss ph: 0.15398992764453093
	Repetition -- Loss ws: 0.09609452458719413
	Repetition -- Loss ss: 0.3139335254828135
	Comprehension -- Loss ph: 0.10174027227693135
	Comprehension -- Loss ws: 0.08856318487889236
	Comprehension -- Loss ss: 0.09075367015217327
	Production -- Loss ph: 0.10324317836978783
	Production -- Loss ws: 0.08277741681784391
	Production -- Loss ss: 0.3007949549208085
	Repetition_Word -- NA
	Comprehension_Word -- NA
	Production_Word -- NA
Epoch 10
	Repetition -- Loss ph: 0.1374197501440843
	Repetition -- Loss ws: 0.11333197837074598
	Repetition -- Loss ss: 0.3300779587527116
	Comprehension -- Loss ph: 0.10157636635005474
	Comprehension -- Loss ws: 0.09869203493412998
	Comprehension -- Loss ss: 0.08279393411137992
	Production -- Loss ph: 0.0910449882565687
	Production -- Loss ws: 0.08354807146514455
	Production -- Loss ss: 0.31731464078029
	Repetition_Word -- NA
	Comprehension_Word -- NA
	Production_Word -- NA
Epoch 15
	Repetition -- Loss ph: 0.13

## Examining model behavior

In [ ]:
import ast

import pandas as pd

import LMInterface    # For reading in the sentences from file
import lm_dataloading # For converting those sentences into a Corpus

from torch.utils.data import DataLoader # For converting the Corpus into a Dataloader
from torch import load                  # Loading pre-trained model
from pathlib import Path                # Path stuff


def load_nonword_memory_lists(language, interface,
                               nonword_legality = 'legal',
                               num_lists = 500,
                               list_length = 4,
                               task_weights = {'repetition' : 1},
                               sep = '\t'):
  """
  """
  # Read in nonword memory lists as a list of dictionaries
  memory_list_dir = Path.cwd().joinpath(f'language/{language}/nonword_memory_lists_{nonword_legality}_length{list_length}_n{num_lists}.tsv')
  memory_lists = LMInterface.utterances_from_file(utterances_file = memory_list_dir, sep = sep)

  # Generate Corpus object of the memory lists
  corpus_memory_lists = lm_dataloading.Corpus(utterances = [l['utterance_lst'] for l in memory_lists],
                                              utterance_infos = [l.to_dict() for l in memory_lists],
                                              interface = interface,
                                              task_weights = task_weights)

  # Generate dataloader
  dataloader_nwml = DataLoader(corpus_memory_lists,
                               batch_size = None)

  return dataloader_nwml

def load_noun_memory_lists(language, interface,
                           num_lists = 500,
                           list_length = 4,
                           task_weights = {'repetition' : 1},
                           sep = '\t'):
  """
  """
  # Read in noun memory lists as a list of dictionaries
  memory_list_dir = Path.cwd().joinpath(f'language/{language}/noun_memory_lists_length{list_length}_n{num_lists}.tsv')
  memory_lists = LMInterface.utterances_from_file(utterances_file = memory_list_dir, sep = sep)

  # Generate Corpus object of the memory lists
  corpus_memory_lists = lm_dataloading.Corpus(utterances = [l['utterance_lst'] for l in memory_lists],
                                              utterance_infos = [l.to_dict() for l in memory_lists],
                                              interface = interface,
                                              task_weights = task_weights)

  # Generate dataloader
  dataloader_nml = DataLoader(corpus_memory_lists,
                              batch_size = None)

  return dataloader_nml

def load_scrambled_sentences(language, interface,
                             task_weights = {'repetition' : 1},
                             sep = '\t'):
  """
  """
  # Read in scrambled sentences as a list of dictionaries
  scrambled_sentence_dir = Path.cwd().joinpath(f'language/{language}/scrambled_sentences.tsv')
  scrambled_sentences = LMInterface.utterances_from_file(utterances_file = scrambled_sentence_dir, sep = sep)

  # Generate Corpus object of scrambled sentences
  corpus_scrambled_sentences = lm_dataloading.Corpus(utterances = [l['utterance_lst'] for l in scrambled_sentences],
                                                     utterance_infos = [s.to_dict() for s in scrambled_sentences],
                                                     interface = interface,
                                                     task_weights = task_weights)

  # Generate dataloader
  dataloder_scrambled = DataLoader(corpus_scrambled_sentences,
                                   batch_size = None)

  return dataloder_scrambled

def load_lexsyn_violations(language, interface,
                           violation = 'vb_inn',
                           task_weights = {'repetition' : 1},
                           sep = '\t'):
  """
  In this case, we also want to check whether the specific sentence being changed is in the training set
  """
  # Read in lexico-syntactic violations as a list of dictionaries
  lexsyn_violations_dir = Path.cwd().joinpath(f'language/{language}/sentence_{violation}.tsv')
  lexsyn_violations = LMInterface.utterances_from_file(utterances_file = lexsyn_violations_dir, sep = sep)

  # Generate Corpus object of lexico-syntactic violations
  corpus_lexsyn_violations = lm_dataloading.Corpus(utterances = [l['utterance_lst'] for l in lexsyn_violations],
                                                   utterance_infos = [s.to_dict() for s in lexsyn_violations],
                                                   interface = interface,
                                                   task_weights = task_weights)

  # Generate dataloader
  dataloader_lexsyn_violations = DataLoader(corpus_lexsyn_violations,
                                            batch_size = None)
  return dataloader_lexsyn_violations

def load_model_env(model_info_path,
                   task_weights = {'repetition' : 1, 'comprehension' : 1, 'production' : 1},
                   sep = ','):
  """
  Read in training and testing dataloaders for a mdoel
  """
  training_env_path = model_info_path.joinpath('train_corpus_info.csv')
  testing_env_path = model_info_path.joinpath('test_corpus_info.csv')

  training_env = LMInterface.utterances_from_file(utterances_file = training_env_path, sep = sep)
  testing_env = LMInterface.utterances_from_file(utterances_file = testing_env_path, sep = sep)

  corpus_sentences_training = lm_dataloading.Corpus(utterances = [sentence['sentence_lst'] for sentence in training_env],
                                                    utterance_infos = training_env,
                                                    interface = interface,
                                                    task_weights = task_weights)
  corpus_sentences_testing = lm_dataloading.Corpus(utterances = [sentence['sentence_lst'] for sentence in testing_env],
                                                   utterance_infos = testing_env,
                                                   interface = interface,
                                                   task_weights = task_weights)

  dataloader_sentences_training = DataLoader(corpus_sentences_training,
                                             batch_size = None)
  dataloader_sentences_testing = DataLoader(corpus_sentences_testing,
                                            batch_size = None)

  return dataloader_sentences_training, dataloader_sentences_testing, training_env, testing_env

def load_model(interface, model_info_path, trained_epochs):
  """
  """
  model_id = model_info_path.stem

  # Load in model
  # -- Access model log to get info about trained model, identifying which model is trained trained_epochs times
  # -- If multiple rows selected (i.e. model saved multiple times at same epoch) select last row
  model_log_path = model_info_path.joinpath('model_log.csv')
  model_log = pd.read_csv(model_log_path)
  assert trained_epochs in model_log['trained_epochs'].values # Assert the model is trained
  model_info = model_log.loc[model_log['trained_epochs'] == trained_epochs] # Here we select the model
  if model_info.shape[0] > 1:
    model_info = model_info.iloc[-1]

  # -- Create model and read in the weights
  model, optimizer, scheduler = LichtheimMemory.inst_LM(interface = interface,
                                                        phh1_size = model_info['phh1_size'].item(),
                                                        wsh1_size = model_info['wsh1_size'].item(), wsh2_size = model_info['wsh2_size'].item(),
                                                        ssh1_size = model_info['ssh1_size'].item(), ssh2_size = model_info['ssh2_size'].item(), ssh3_size = model_info['ssh3_size'].item(),
                                                        seed = model_info['seed'].item(),
                                                        lr = model_info['lr'].item(), weight_decay = model_info['weight_decay'].item(), lr_steprate = model_info['lr_steprate'].item(), lr_gamma = model_info['lr_gamma'].item(),
                                                        model_id = model_id)

  # -- Identify the path to the trained model
  try:
    model_trained_path = Path(model_info['model_object_path'].item())
  except AttributeError:
    model_trained_path = Path(model_info['model_object_path'])
  model.load_state_dict(load(model_trained_path))

  return model

In [ ]:
import lm_behavior
import lm_logging

def test_model_performance(model, dataloader, interface, behavior_type,
                           meta_info = {'trained_epochs' : None}):
  """
  Test model performance on a generic task, with input passed in through datalaoder object
  """
  # Get losses and behaviors
  behaviors, loss = lm_behavior.test_epoch(model = model,
                                           dataloader = dataloader)

  # Package output of behavior
  behaviors = lm_logging.package_all_output(behavior = behaviors,
                                            interface = interface)

  # Log all losses and behaviors
  lm_logging.log_model_behavior(model = model,
                                behavior = behaviors,
                                trained_epochs = meta_info['trained_epochs'],
                                behavior_type = behavior_type,
                                meta_info = meta_info)
  lm_logging.log_model_loss(model = model,
                            loss = loss,
                            trained_epochs = meta_info['trained_epochs'],
                            behavior_type = behavior_type,
                            meta_info = meta_info)

def test_multiple_model_performances(model, task_to_dataloaders, interface,
                                     meta_info = {'trained_epochs' : None}):
  """
  Tasks to dataloaders a dictionary of tasks and the dataloaders associated
  with those tasks
  """
  for task in task_to_dataloaders:
    test_model_performance(model = model,
                           dataloader = task_to_dataloaders[task],
                           interface = interface,
                           behavior_type = task,
                           meta_info = meta_info)

In [ ]:
import pandas as pd

from pathlib import Path

def unique(sequence):
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def load_task_dataloaders(interface, model_path,
                          memory_list_interface = None,
                          tasks = ['train', 'test', 'scrambled_sentence', 'nonword_list', 'single_nonword_repetition',
                                   'lexsyn_violation_vb_trn', 'lexsyn_violation_vb_int', 'lexsyn_violation_vb_inn', 'lexsyn_violation_snb_inn', 'lexsyn_violation_pvnb_inn'],
                          nonword_legality = 'legal'):
  """
  """
  # -- Model environment needs to be loaded once
  dataloader_sentences_training, dataloader_sentences_testing, training_env, testing_env = load_model_env(model_info_path = model_path)

  task_to_dataloaders = {}
  if 'train' in tasks:
    task_to_dataloaders.update({'train' : dataloader_sentences_training})
  if 'test' in tasks:
    task_to_dataloaders.update({'test' : dataloader_sentences_testing})
  if 'scrambled_sentence' in tasks:
    dataloader_scrambled_sentences = load_scrambled_sentences(language = interface.language_dir.stem,
                                                              interface = interface)
    task_to_dataloaders.update({'scrambled_sentence' : dataloader_scrambled_sentences})
  if 'noun_list' in tasks:
    dataloader_noun_list = load_noun_memory_lists(language = interface.language_dir.stem,
                                                  interface = interface)
    task_to_dataloaders.update({'noun_list' : dataloader_noun_list})
  if 'nonword_list' in tasks:
    dataloader_nonword_list = load_nonword_memory_lists(language = memory_list_interface.language_dir.stem,
                                                        interface = memory_list_interface,
                                                        nonword_legality = nonword_legality)
    task_to_dataloaders.update({'nonword_list' : dataloader_nonword_list})
  if 'single_nonword_repetition' in tasks:
    dataloader_single_nonword_list = load_nonword_memory_lists(language = memory_list_interface.language_dir.stem,
                                                               interface = memory_list_interface,
                                                               nonword_legality = nonword_legality,
                                                               list_length = 1)
    task_to_dataloaders.update({'single_nonword_repetition' : dataloader_single_nonword_list})
  if 'lexsyn_violation_vb_trn' in tasks:
    dataloader_lexsyn_vb_trn = load_lexsyn_violations(language = interface.language_dir.stem,
                                                      interface = interface,
                                                      violation = 'vb_trn')
    task_to_dataloaders.update({'lexsyn_violation_vb_trn' : dataloader_lexsyn_vb_trn})
  if 'lexsyn_violation_vb_int' in tasks:
    dataloader_lexsyn_vb_int = load_lexsyn_violations(language = interface.language_dir.stem,
                                                      interface = interface,
                                                      violation = 'vb_int')
    task_to_dataloaders.update({'lexsyn_violation_vb_int' : dataloader_lexsyn_vb_int})
  if 'lexsyn_violation_vb_inn' in tasks:
    dataloader_lexsyn_vb_inn = load_lexsyn_violations(language = interface.language_dir.stem,
                                                      interface = interface,
                                                      violation = 'vb_inn')
    task_to_dataloaders.update({'lexsyn_violation_vb_inn' : dataloader_lexsyn_vb_inn})
  if 'lexsyn_violation_snb_inn' in tasks:
    datalaoder_lexsyn_snb_inn = load_lexsyn_violations(language = interface.language_dir.stem,
                                                       interface = interface,
                                                       violation = 'snb_inn')
    task_to_dataloaders.update({'lexsyn_violation_snb_inn' : datalaoder_lexsyn_snb_inn})
  if 'lexsyn_violation_pvnb_inn' in tasks:
    dataloader_lexsyn_pvnb_inn = load_lexsyn_violations(language = interface.language_dir.stem,
                                                        interface = interface,
                                                        violation = 'pvnb_inn')
    task_to_dataloaders.update({'lexsyn_violation_pvnb_inn' : dataloader_lexsyn_pvnb_inn})

  return task_to_dataloaders

def load_and_test_models(model_id, interface,
                         trained_model_epochs = None,
                         memory_list_interface = None,
                         tasks = ['train', 'test', 'scrambled_sentence', 'noun_list', 'nonword_list', 'single_nonword_repetition',
                                 'lexsyn_violation_vb_trn', 'lexsyn_violation_vb_int', 'lexsyn_violation_vb_inn', 'lexsyn_violation_snb_inn', 'lexsyn_violation_pvnb_inn'],
                         nonword_legality = 'legal',
                         trained_epochs_key = 'trained_epochs',
                         report_progress = True):
  """
  """
  model_path = Path.cwd().joinpath(f'model_info/{model_id}')

  # Get all models trained across all epochs, or keep the target trained models already in trained_model_epochs
  model_training_info = pd.read_csv(model_path.joinpath('model_log.csv'))
  if not trained_model_epochs:
    trained_model_epochs = unique(list(model_training_info[trained_epochs_key]))

  # Read in all dataloaders for all tasks
  task_to_dataloaders = load_task_dataloaders(interface = interface, model_path = model_path,
                                              memory_list_interface = memory_list_interface,
                                              tasks = tasks,
                                              nonword_legality = nonword_legality)

  # -- Each model is loaded and then tested on different sets of sentences
  for trained_epochs in trained_model_epochs:
    if report_progress: print(f'{trained_epochs} -- {model_id}')
    LM = load_model(interface = interface,
                    model_info_path = model_path,
                    trained_epochs = trained_epochs)

    test_multiple_model_performances(model = LM, task_to_dataloaders = task_to_dataloaders,
                                     interface = interface, meta_info = {'trained_epochs' : trained_epochs})

## Examine specific model behavior

In [ ]:
model_id = "lm_129_2023-04-18_853720c5-c833-4e01-9e37-20e8960a80da"

nonword_legality = 'legal'
nonword_memory_list_interface = LMInterface.LMInterface(language_dir = language_dir,
                                                        phonology_filestem = f'phonology_artificial_{nonword_legality}_nonwords')

#tasks = ['train', 'test', 'scrambled_sentence', 'nonword_list', 'single_nonword_repetition', 'lexsyn_violation_vb_trn', 'lexsyn_violation_vb_int', 'lexsyn_violation_vb_inn', 'lexsyn_violation_snb_inn', 'lexsyn_violation_pvnb_inn']
#tasks = ['lexsyn_violation_vb_trn', 'lexsyn_violation_vb_int', 'lexsyn_violation_vb_inn', 'lexsyn_violation_snb_inn', 'lexsyn_violation_pvnb_inn']
tasks = ['noun_list']

load_and_test_models(model_id = model_id,
                     interface = interface,
                     trained_model_epochs = ['199'],
                     memory_list_interface = nonword_memory_list_interface,
                     tasks = tasks,
                     nonword_legality = nonword_legality)

199 -- lm_129_2023-04-18_853720c5-c833-4e01-9e37-20e8960a80da
